# RiscV

Serv core - smallest riscv? https://github.com/olofk/serv.git  LUT FF flip flop
https://github.com/enjoy-digital/litex litex supports it

https://github.com/YosysHQ/picorv32
https://github.com/BrunoLevy/learn-fpga
femtorv quARK https://github.com/BrunoLevy/learn-fpga/blob/master/FemtoRV/RTL/PROCESSOR/femtorv32_quark.v

https://github.com/rust-embedded/riscv low level access to risc v processor in rust
https://popovicu.com/posts/bare-metal-rust-risc-v-with-dynamic-memory/
Kani + aeneas? https://github.com/AeneasVerif/aeneas


https://crates.io/crates/rrs-lib
https://crates.io/crates/riscv
https://crates.io/crates/rvemu less stars

https://github.com/black-parrot/black-parrot  "Other mappings, such the multiplier to DSP48, could be improved." Something zach mentioned in a mocha ideation m,eeting
https://github.com/chipsalliance/dromajo riscv simulator for rtl cosim
https://bellard.org/tinyemu/ 


In [11]:
%%file /tmp/risc.s
.global start
_start:
    mov r0, #1
    mov r1, #2
    add r2, r0, r1
    mov r3, #0
    swi 0


Overwriting /tmp/risc.s


In [3]:
! riscv64-linux-gnu-as -o /tmp/risc.o /tmp/risc.s

/bin/bash: line 1: riscv64-linux-gnu-as: command not found


In [12]:
! ~/Downloads/riscv_toolchain/bin/riscv32-unknown-elf-as -o /tmp/risc.o /tmp/risc.s

In [ ]:
%%file /tmp/rustemu.rs
use rrs_lib::{HartState, MemAccessSize, Memory};
use rrs_lib::memories::VecMemory;
use rrs_lib::instruction_executor::{InstructionExecutor, InstructionException};
use rrs_lib::instruction_string_outputter::InstructionStringOutputter;

fn simulate_riscv() {
  let mut hart = HartState::new();
  // Memory contains these instructions:
  // lui x2, 0x1234b
  // lui x3, 0xf387e
  // add x1, x2, x3
  let mut mem = VecMemory::new(vec![0x1234b137, 0xf387e1b7, 0x003100b3]);

  hart.pc = 0;

  // InstructionExecutor implements IntructionProcessor. The step function calls
  // process_instruction internally and handles things like updating the PC.
  let mut executor = InstructionExecutor {
      hart_state: &mut hart,
      mem: &mut mem,
  };

  // Execute first instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[2], 0x1234b000);

  // Execute second instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[3], 0xf387e000);

  // Execute third instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[1], 0x05bc9000);

  // Memory only contains three instructions so next step will produce a fetch error
  assert_eq!(executor.step(), Err(InstructionException::FetchError(0xc)));
}

fn output_disass<M: Memory>(executor: &mut InstructionExecutor<M>) {
  let mut outputter = InstructionStringOutputter { insn_pc: executor.hart_state.pc };
  let insn_bits = executor.mem.read_mem(executor.hart_state.pc, MemAccessSize::Word).unwrap();
  println!("{}", rrs_lib::process_instruction(&mut outputter, insn_bits).unwrap());
}

# Interrupts
https://www.cs.ox.ac.uk/tom.melham/pub/Kroening-2015-EVL.pdf cbmc

goto-instrument --isr interrupt_1 --dump-c

https://users.cs.utah.edu/~regehr/papers/tv06.pdf Regehr

https://sos-vo.org/sites/sos-vo.org/files/sos_files/Interrupts_in_OS_code_lets_reason_about_them_Yes_this_means_concurrency.pdf

In [1]:
%%file /tmp/int1.c
#include <assert.h>

int x = 0;

void interrupt_1() {
    x += 1;
}

int main (int argc, char *argv[]) {
    x = 0;
    assert(x == 0);
    return x;
}

Writing /tmp/int1.c


In [ ]:
! c

# Bootlading
https://news.ycombinator.com/item?id=41161580  	A cryptographically secure bootloader for RISC-V in Rust (codethink.co.uk)
https://www.codethink.co.uk/articles/2024/secure_bootloader/ 
riscv vector cryptography extension

I have old notes somewhere

PUF https://en.wikipedia.org/wiki/Physical_unclonable_function

https://arstechnica.com/security/2024/07/secure-boot-is-completely-compromised-on-200-models-from-5-big-device-makers/

https://github.com/vvaltchev/tilck
http://github.com/oreboot/oreboot

uboot


https://github.com/river-li/awesome-uefi-security

open titan 
https://github.com/lowRISC/opentitan.git
https://opentitan.org/book/doc/security/specs/attestation/ hardware attestation
https://opentitan.org/book/sw/device/silicon_creator/rom/index.html ROM is first stage boot

https://docs.arduino.cc/retired/hacking/software/Bootloader/ arduino bootloader

Running risc v. Emulating m,icrocontroller?


privlenged architexcture riscv
https://riscv.org/wp-content/uploads/2017/05/riscv-privileged-v1.10.pdf

opensbi https://github.com/riscv-software-src/opensbi

acronyms out the wing wong
PMP - physical memory protection
TPM

PCR registers. Extend and hash. Makes a registery merkle list thing.

bootkeeper https://arxiv.org/pdf/1903.12505

[1] Oleksandr Bazhaniuk, John Loucaides, Lee Rosenbaum, Mark R Tuttle, and Vincent Zimmer. 2015. Symbolic execution for BIOS security. In 9th USENIX Workshop
on Offensive Technologies (WOOT ’15).
[2] Jonathan Brossard. 2012. Hardware backdooring is practical. BlackHat, Las Vegas,
USA (2012).
[3] Danilo Bruschi, Lorenzo Cavallaro, Andrea Lanzi, and Mattia Monga. 2005. Replay attack in TCG specification and solution. In Proceeding of the 21st Annual
Computer Security Applications Conference (ACSAC ’05). IEEE, 127–137.
[4] John Butterworth, Corey Kallenberg, Xeno Kovah, and Amy Herzog. 2013. BIOS
Chronomancy: Fixing the core root of trust for measurement. In Proceedings of
the 2013 ACM SIGSAC conference on Computer & Communications Security (CCS
’13). ACM, 25–36.


https://wiki.archlinux.org/title/Trusted_Platform_Module linux access to tpm

https://surface.syr.edu/cgi/viewcontent.cgi?article=1182&context=eecs_techreports sable constable. isabelle verified bootloader
skinit instruction. Start process with hash in a pcr and completely fresh isolated? What's the point though? If the insecure part has any effect. If it doesn't why was it there?
Dynamic

https://google.github.io/tpm-js/ javascript tpm in browser

## Attacks and Glitching and Fault injection

Chipwhisperer http://wiki.newae.com/V4:Tutorial_A2_Introduction_to_Glitch_Attacks_(including_Glitch_Explorer)

https://www.youtube.com/watch?v=XvGcQgx9Jg8&ab_channel=BlackHat breaking secure bootloaders blackhat - bootloader has a bad parser in it

https://limitedresults.com/2019/09/pwn-the-esp32-secure-boot/

Glitching / fault injection. Look at code, correlated power draw? Wiggle voltage at opporuitne times. Or lasers if you're fancy

https://github.com/janvdherrewegen/bootl-attacks
https://www.youtube.com/watch?v=T1NHrzhtqzA&ab_channel=IACR

https://machiry.github.io/files/glitchresistor.pdf glitch resistor. Modellign glitching and llvm plugin for resistance
Redundant checks because you aren't sure insturctions weren't skipped.

https://static.linaro.org/connect/lvc21f/presentations/LVC21F-116.pdf
https://ieeexplore.ieee.org/document/8167709
https://ibex-core.readthedocs.io/en/latest/03_reference/security.html

https://github.com/ucsb-seclab/BootStomp
https://dl.acm.org/doi/10.5555/3241189.3241251

https://www.riscure.com/publication/fault-mitigation-patterns/

## AB Update
"A/B" update https://opentitan.org/book/doc/security/logical_security_model/index.html?highlight=A%2FB#firmware-update-considerations So the stage0 bootloader is in charge of checking the sanctity of the new bootimage and picking the newest one

https://source.android.com/docs/core/ota/ab


https://source.android.com/docs/core/ota/ab/ab_implement

https://bootlin.com/pub/conferences/2022/elce/opdenacker-implementing-A-B-system-u[…]boot/opdenacker-implementing-A-B-system-updates-with-u-boot.pdf

Signature
decryption
other things?
rboot https://github.com/raburton/rboot/blob/master/rboot.c


# Qemu
sudo apt-get update
sudo apt-get install opensbi qemu-system-misc u-boot-qemu
https://www.qemu.org/docs/master/system/target-riscv.html

https://popovicu.com/posts/bare-metal-programming-risc-v/

QEMU -bios flag
0x80000000
qemu loads up opensbi

https://github.com/qemu/qemu/blob/master/hw/riscv/virt.c#L78-L99 memory map

```C
static const MemMapEntry virt_memmap[] = {
    [VIRT_DEBUG] =        {        0x0,         0x100 },
    [VIRT_MROM] =         {     0x1000,        0xf000 },
    [VIRT_TEST] =         {   0x100000,        0x1000 },
    [VIRT_RTC] =          {   0x101000,        0x1000 },
    [VIRT_CLINT] =        {  0x2000000,       0x10000 },
    [VIRT_ACLINT_SSWI] =  {  0x2F00000,        0x4000 },
    [VIRT_PCIE_PIO] =     {  0x3000000,       0x10000 },
    [VIRT_PLATFORM_BUS] = {  0x4000000,     0x2000000 },
    [VIRT_PLIC] =         {  0xc000000, VIRT_PLIC_SIZE(VIRT_CPUS_MAX * 2) },
    [VIRT_APLIC_M] =      {  0xc000000, APLIC_SIZE(VIRT_CPUS_MAX) },
    [VIRT_APLIC_S] =      {  0xd000000, APLIC_SIZE(VIRT_CPUS_MAX) },
    [VIRT_UART0] =        { 0x10000000,         0x100 },
    [VIRT_VIRTIO] =       { 0x10001000,        0x1000 },
    [VIRT_FW_CFG] =       { 0x10100000,          0x18 },
    [VIRT_FLASH] =        { 0x20000000,     0x4000000 },
    [VIRT_IMSIC_M] =      { 0x24000000, VIRT_IMSIC_MAX_SIZE },
    [VIRT_IMSIC_S] =      { 0x28000000, VIRT_IMSIC_MAX_SIZE },
    [VIRT_PCIE_ECAM] =    { 0x30000000,    0x10000000 },
    [VIRT_PCIE_MMIO] =    { 0x40000000,    0x40000000 },
    [VIRT_DRAM] =         { 0x80000000,           0x0 },
};

/* PCIe high mmio is fixed for RV32 */
#define VIRT32_HIGH_PCIE_MMIO_BASE  0x300000000ULL
#define VIRT32_HIGH_PCIE_MMIO_SIZE  (4 * GiB)

/* PCIe high mmio for RV64, size is fixed but base depends on top of RAM */
#define VIRT64_HIGH_PCIE_MMIO_SIZE  (16 * GiB)
```

In [ ]:
%%file /tmp/

In [ ]:
%%bash
riscv32-none-elf-as -o /tmp/int1.o /tmp/int1.c
riscv32-none-elf-ld -o /tmp/int1 /tmp/int1.o
qemu-system-riscv32 -nographic -machine spike_v1.10 -kernel /tmp/int1

In [2]:
c!qemu-system-riscv32

qemu-system-riscv32: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [17]:
%%file /tmp/minimal.ld
MEMORY {
    dram_space (rwx): ORIGIN = 0x80000000, LENGTH = 0x128
}

SECTIONS {
    .text : {
        minimal.o(.text.minimal)
    } > dram_space
    }

Overwriting /tmp/minimal.ld


In [16]:
%%file /tmp/minimal.s
.global _start
.section .text.bios

_start: 
    addi a0, x0, 0x68
    li a1, 0x10000000
    sb a0, (a1) #'h'

loop: j loop


Overwriting /tmp/minimal.s


In [25]:
%%bash
PATH=~/Downloads/riscv_toolchain/bin:$PATH
riscv32-unknown-elf-as -o /tmp/minimal.o /tmp/minimal.s
#riscv32-unknown-elf-objdump -d /tmp/minimal.o
riscv32-unknown-elf-ld -T /tmp/minimal.ld -o /tmp/minimal /tmp/minimal.o
#qemu-system-riscv32 -nographic -machine spike_v1.10 -bios /tmp/minimal

riscv32-unknown-elf-ld: cannot find minimal.o: No such file or directory


CalledProcessError: Command 'b'PATH=~/Downloads/riscv_toolchain/bin:$PATH\nriscv32-unknown-elf-as -o /tmp/minimal.o /tmp/minimal.s\n#riscv32-unknown-elf-objdump -d /tmp/minimal.o\nriscv32-unknown-elf-ld -T /tmp/minimal.ld -o /tmp/minimal /tmp/minimal.o\n#qemu-system-riscv32 -nographic -machine spike_v1.10 -bios /tmp/minimal\n'' returned non-zero exit status 1.

In [22]:
! ~/Downloads/riscv_toolchain/bin/objdump -d /tmp/minimal.o

/bin/bash: line 1: /home/philip/Downloads/riscv_toolchain/bin/objdump: No such file or directory


## Esp32
https://docs.espressif.com/projects/esp-idf/en/latest/esp32/get-started/linux-macos-setup.html#get-started-get-esp-idf
vscode esp-idf


```
cd ~/esp
cp -r $IDF_PATH/examples/get-started/hello_world .
cd ~/esp/hello_world
idf.py set-target esp32
idf.py menuconfig
idf.py build
idf.py -p PORT flash
```

# Userland load

In [25]:
%%file /tmp/code.s
.global _start

_start:
    mov $42, %rax
    ret

Writing /tmp/code.s


In [28]:
%%bash
as -o /tmp/code.o /tmp/code.s
objcopy -O binary /tmp/code.o /tmp/code.bin
hexdump -v -e '"0x" 1/1 "%02X" ", "' /tmp/code.bin

0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3, 

In [21]:
%%file /tmp/simploader.c
#include <stdio.h>
#include <stdint.h>

unsigned char code[] __attribute__ ((section (".text"))) = {
    0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3
};
int main(){
    int (*func)() = (int (*)())code;
    int retcode = func();
    
    printf("retcode %d\n", retcode);
    return 0;
}

Overwriting /tmp/simploader.c


In [25]:
!frama-c -wp /tmp/simploader.c

[kernel] Parsing /tmp/simploader.c (with preprocessing)
[kernel:typing:incompatible-pointer-types] /tmp/simploader.c:8: Warning: 
  casting function from unsigned char *
[kernel:annot:missing-spec] FRAMAC_SHARE/libc/stdio.h:211: Warning: 
  Neither code nor explicit exits and terminates for function printf_va_1,
   generating default clauses. See -generated-spec-* options for more info
[wp] /tmp/simploader.c:7: Warning: 
  In 'main', no 'calls' specification for statement(s) on line(s): line 9, 
  Assuming that they can call 'main'
[wp] Warning: Missing RTE guards
[wp] /tmp/simploader.c:9: Warning: Missing 'calls' for default behavior
[wp] /tmp/simploader.c:9: Warning: 
  Unknown callee, considering non-terminating call
[wp] /tmp/simploader.c:9: Warning: 
  Missing decreases clause on recursive function main, call must be unreachable
[wp] /tmp/simploader.c:8: Warning: 
  Cast with incompatible pointers types (source: uint8*) (target: void**)
[wp] User Error: Prover 'Alt-Ergo' not found

In [20]:
%%file /tmp/simploader.c
#include <stdio.h>
#include <stdint.h>
#include <sys/mman.h>
#include <string.h>
#include <unistd.h>

unsigned char code[] = {
    //0x90, 0x90, 0xC3 /// (NOP; NOP; RET)
    0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3
    // Put code here
};
int main(){
    size_t pagesize = sysconf(_SC_PAGESIZE); //getpagesize();
    void* exec_mem = mmap(NULL, pagesize, PROT_WRITE | PROT_EXEC, MAP_ANON | MAP_PRIVATE, -1, 0); // Hmm. shouldn't cbmc have noted I should have checked for fail here?
    memcpy(exec_mem, code, sizeof(code));

    int (*func)() = (int (*)())exec_mem;
    int retcode = func();
    
    printf("retcode %d\n", retcode);
    munmap(exec_mem, pagesize);
    return 0;
}

Writing /tmp/simploader.c


In [44]:
!gcc -Wall -Wextra -Werror -o /tmp/simploader /tmp/simploader.c && /tmp/simploader

/tmp/ccXxw4Nh.s: Assembler messages:
/tmp/ccXxw4Nh.s:4: Warning: ignoring changed section attributes for .text
retcode 42


In [ ]:
!

In [35]:
!cbmc /tmp/simploader.c --pointer-check

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking simploader
Generating GOTO Program
Adding CPROVER library (x86_64)


Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking
Passing problem to propositional reduction
converting SSA
Running propositional reduction
SAT checker: instance is SATISFIABLE
Running propositional reduction
SAT checker inconsistent: instance is UNSATISFIABLE

** Results:
/tmp/simploader.c function main
[main.precondition_instance.1] line 16 memcpy src/dst overlap: SUCCESS
[main.precondition_instance.2] line 16 memcpy source region readable: SUCCESS
[main.precondition_instance.3] line 16 memcpy destination region writeable: FAILURE
[main.pointer_dereference.1] line 19 dereference failure: pointer NULL in func: SUCCESS
[main.pointer_dereference.2] line 19 dereference failure: pointer invalid in func: SUCCESS
[main.pointer_dereference.3] line 19 dereference failure: deallocated dynamic object in func: SUCCESS
[main.pointer_dereference.4] line 19 dereference failure: dead object in func: SUCCESS
[main.pointer_dereference.5

In [ ]:
%%file /tmp/simploader.c

int code[] = {
    // Put code here
}
// int codes[3][1024] = {{},{}, {}}
int crc = 0xdeadbeef;
int pub_auth_key;

int auth(int* code, int size, int key ){

}

int decrypt_buffer[1024];
int decrypt(int , int size, int* buffer){

}

typedef struct config_t{
} config_t;

int main(){
    check_config();
    auth(code, sizeof(code), sig);
    decrypt(code, sizeof(code), decrypt_buffer);
    (void* code())();
}



In [ ]:
!cbmc /tmp/simploader.c

# Simulation
cocotb
verilator https://verilator.org/guide/latest/install.html#git-install
ibex verilator https://github.com/lowRISC/ibex/blob/master/examples/simple_system/README.md


# FPGA

https://pulp-platform.org/docs/coscup2024/phsauter_coscup24.pdf  Designing Linux-capable systems using open EDA tools Pulp
https://github.com/pulp-platform
Used for ETH Zürich VLSI 2 lecture starting 2025 Croc soc
github.com/pulp-platform/croc

place and route


https://github.com/chipsalliance/Surelog  SystemVerilog 2017 Pre-processor, Parser, Elaborator, UHDM Compiler. Provides IEEE Design/TB C/C++ VPI and Python AST API.
https://github.com/bespoke-silicon-group/bsg_sv2v/tree/master sysverilog to verilog
https://github.com/chipsalliance/rocket-chip

olof kindgren
fusesoc https://github.com/olofk/fusesoc package manager 
edalizer - port acorss different guys? https://github.com/olofk/edalize


iverilog

https://projectf.io/posts/verilog-sim-verilator-sdl/ verilator to sdl


Lakeroad / sam coward things
It is an interesting and possibly noble endeavor to try and make open hardware / fpga compiling as good as open software

Get out some intermediate rwepresentation from yosys.
Emulate stuff via iverilog
nmigen

https://github.com/YosysHQ/oss-cad-suite-build#installation
https://yowasp.org/ pip install yowasp-yosys

ffirtl https://github.com/chipsalliance/firrtl

RTLIL is yosys internal IR. Text rep

Lakeroad https://arxiv.org/abs/2401.16526
https://github.com/uwsampl/lakeroad
https://github.com/uwsampl/lakeroad/blob/main/bin/verilog_to_racket.py yosys btor output


Could also root around in rachit's stuff

https://github.com/f4pga/f4pga-arch-defs

In [6]:
%%file /tmp/counter.v
module counter(
    input wire clk,
    input wire rst,
    output reg [7:0] count
);

always @(posedge clk or posedge rst) begin
    if (rst) begin
        count <= 8'b0;
    end else begin
        count <= count + 1;
    end
end

endmodule

Overwriting /tmp/counter.v


In [8]:
%%bash
#output btor2
# ERROR: Unsupported cell type $_DFF_PP0_ for cell counter.$auto$ff.cc:266:slice$89 -- please run `async2sync; dffunmap` or `clk2fflogic` before `write_btor`.
yosys -p 'read_verilog /tmp/counter.v; synth -top counter; async2sync; dffunmap; write_btor /tmp/counter.btor'


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42+40 (git sha1 a739e21a5, clang++ 14.0.0-1ubuntu1.1 -fPIC -Os)

-- Running command `read_verilog /tmp/counter.v; synth -top counter; async2sync; dffunmap; write_btor /tmp/counter.btor' --

1. Executing Verilog-2005 frontend: /tmp/counter.v
Parsing Verilog input from `/tmp/counter.v' to AST representation.
Generating RTLIL representation for module `\counter'.
Successfully finished Verilog frontend.

2. Executing SYNTH pass.

2.1. Executing HIERARCHY pass (managing design hierarchy).

2.1.1. Analyzing design hierarchy..
Top module:  \counter

2.1.2. Analyzing design hierarchy..
Top m


2.6.4. Executing OPT_REDUCE pass (consolidate $*mux and $reduce_* inputs).
  Optimizing cells in module \counter.
Performed a total of 0 changes.

2.6.5. Executing OPT_MERGE pass (detect identical cells).
Finding identical cells in module `\counter'.
Removed a total of 0 cells.

2.6.6. Executing OPT_DFF pass (perform DFF optimizations).

2.6.7. Executing OPT_CLEAN pass (remove unused cells and wires).
Finding unused cells or wires in module \counter..

2.6.8. Executing OPT_EXPR pass (perform const folding).
Optimizing module counter.

2.6.9. Finished OPT passes. (There is nothing left to do.)

2.7. Executing FSM pass (extract and optimize FSM).

2.7.1. Executing FSM_DETECT pass (finding FSMs in design).

2.7.2. Executing FSM_EXTRACT pass (extracting FSM from design).

2.7.3. Executing FSM_OPT pass (simple optimizations of FSMs).

2.7.4. Executing OPT_CLEAN pass (remove unused cells and wires).
Finding unused cells or wires in module \counter..

2.7.5. Executing FSM_OPT pass (simple op

In [19]:
import boolector

ModuleNotFoundError: No module named 'boolector'

In [18]:
import bitwuzla
tm = bitwuzla.TermManager()
options = bitwuzla.Options()
p = bitwuzla.Parser(tm, options, "btor2")
p.parse('/tmp/counter.btor')

BitwuzlaException: model checking extensions not supported

In [12]:
%%file /tmp/cpu_test.v

module cpu_test;
    reg clk;
    reg reset;
    wire [7:0] output_signal;

    // Instantiate your CPU module
    /*
    cpu my_cpu (
        .clk(clk),
        .reset(reset),
        .output_signal(output_signal)
    ); */
    assign output_signal = 8'b10101010;

    initial begin
        clk = 0;
        reset = 1;
        #5 reset = 0;
        #1000 $finish;
    end

    always #5 clk = ~clk;

    always @(posedge clk) begin
        // Simple example of ASCII visualization
        $display("Cycle %d: Output signal = %b", $time, output_signal);
    end
endmodule


Overwriting /tmp/cpu_test.v


In [14]:
%%bash
iverilog -o /tmp/cpu_test.vvp /tmp/cpu_test.v
vvp /tmp/cpu_test.vvp

Cycle                    5: Output signal = 10101010
Cycle                   15: Output signal = 10101010
Cycle                   25: Output signal = 10101010
Cycle                   35: Output signal = 10101010
Cycle                   45: Output signal = 10101010
Cycle                   55: Output signal = 10101010
Cycle                   65: Output signal = 10101010
Cycle                   75: Output signal = 10101010
Cycle                   85: Output signal = 10101010
Cycle                   95: Output signal = 10101010
Cycle                  105: Output signal = 10101010
Cycle                  115: Output signal = 10101010
Cycle                  125: Output signal = 10101010
Cycle                  135: Output signal = 10101010
Cycle                  145: Output signal = 10101010
Cycle                  155: Output signal = 10101010
Cycle                  165: Output signal = 10101010
Cycle                  175: Output signal = 10101010
Cycle                  185: Output signal = 10

https://github.com/cocotb/cocotb

Oh. cocotb ius like a testbench. It uses icarus. Hmm. but maybe its an interface to icarus

In [10]:
%%file /tmp/LUT2.v
// What does this do?
`default_nettype none
module LUT2(input A, B, output Z);
    parameter [3:0] init = 4'h0;
    wire [1:0] s1 = B ?     init[ 3:2] :     init[1:0];
    assign Z =      A ?          s1[1] :         s1[0];
endmodule

Overwriting /tmp/LUT2.v


In [6]:
%%file /tmp/yoscript.ys
read_verilog /tmp/LUT2.v

Overwriting /tmp/yoscript.ys


In [2]:
! yowasp-yosys -p read_verilog /tmp/LUT2.v #-s /tmp/yoscript.ys


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42 (git sha1 9b6afcf3f, ccache clang 14.0.0-1ubuntu1.1 -Os -flto -flto)

-- Parsing `/tmp/LUT2.v' using frontend ` -vlog2k' --
ERROR: Can't open input file `/tmp/LUT2.v' for reading: No such file or directory


hmm. Looks like boolector or bitwuzla python bindings might support parsing btor2
https://bitwuzla.github.io/docs/python/interface.html#bitwuzla.Parser.parse
https://boolector.github.io/docs/boolector.html#pyboolector.Boolector.Parse

In [12]:
%%file /tmp/fifo.v

// address generator/counter
module addr_gen 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input en, clk, rst,
	output reg [AWIDTH-1:0] addr
);
	initial addr <= 0;

	// async reset
	// increment address when enabled
	always @(posedge clk or posedge rst)
		if (rst)
			addr <= 0;
		else if (en) begin
			if (addr == MAX_DATA-1)
				addr <= 0;
			else
				addr <= addr + 1;
		end
endmodule //addr_gen

// Define our top level fifo entity
module fifo 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input wen, ren, clk, rst,
	input [7:0] wdata,
	output reg [7:0] rdata,
	output reg [AWIDTH:0] count
);
	// fifo storage
	// sync read before write
	wire [AWIDTH-1:0] waddr, raddr;
	reg [7:0] data [MAX_DATA-1:0];
	always @(posedge clk) begin
		if (wen)
			data[waddr] <= wdata;
		rdata <= data[raddr];
	end // storage

	// addr_gen for both write and read addresses
	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_writer (
		.en     (wen),
		.clk    (clk),
		.rst    (rst),
		.addr   (waddr)
	);

	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_reader (
		.en     (ren),
		.clk    (clk),
		.rst    (rst),
		.addr   (raddr)
	);

	// status signals
	initial count <= 0;

	always @(posedge clk or posedge rst) begin
		if (rst)
			count <= 0;
		else if (wen && !ren)
			count <= count + 1;
		else if (ren && !wen)
			count <= count - 1;
	end

endmodule

Writing /tmp/fifo.v


In [2]:
! yosys /tmp/fifo.v

/bin/bash: line 1: yosys: command not found


# Vivado


So people mostly use TCL and no gui mode?
https://www.xilinx.com/support/documents/sw_manuals/xilinx2022_1/ug835-vivado-tcl-commands.pdf

```
Vivado% help
Vivado% help get_*
```



In [ ]:
%%bash
~/Xilinx/Vivado/2024.1/bin/vivado -mode tcl # interactive
vivado -mode batch -source <your_Tcl_script>

In [1]:
%%file /tmp/counter.v
module counter(
    input clk,
    input reset,
    output reg [3:0] count
);

always @(posedge clk or posedge reset) begin
    if (reset)
        count <= 4'b0000;
    else
        count <= count + 1;
end

endmodule


Overwriting /tmp/counter.v


In [8]:
%%file /tmp/counter_tb.v
`timescale 1ns / 1ps

module counter_tb;
    reg clk;
    reg reset;
    wire [3:0] count;

    // Instantiate the counter module
    counter uut (
        .clk(clk),
        .reset(reset),
        .count(count)
    );

    // Generate a clock signal
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period clock
    end

    // Stimulate the design
    initial begin
        reset = 1;
        #10; // Hold reset for 10ns
        reset = 0;
        #100; // Run simulation for 100ns
        $finish;
    end

    // Monitor the outputs
    initial begin
        $monitor("At time %t, count = %b", $time, count);
    end

endmodule


Writing /tmp/counter_tb.v


In [6]:
%%file /tmp/build.tcl
# non_project_build.tcl

# Set the part number for the target FPGA
set part_name "xc7a100tcsg324-1"
cd /tmp/

# Read the Verilog file
read_verilog "/tmp/counter.v"

# Set the target device
synth_design -top counter -part $part_name

# Run implementation
opt_design
place_design
route_design

# Write the bitstream
#write_bitstream -force "counter.bit"


Overwriting /tmp/build.tcl


In [7]:
!~/Xilinx/Vivado/2024.1/bin/vivado -mode batch -nojournal -source /tmp/build.tcl


****** Vivado v2024.1.1 (64-bit)
  **** SW Build 5094488 on Fri Jun 14 08:57:50 MDT 2024
  **** IP Build 5091682 on Fri Jun 14 16:55:04 MDT 2024
  **** SharedData Build 5094118 on Fri Jun 14 01:09:43 MDT 2024
  **** Start of session at: Fri Aug 30 10:09:44 2024
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /tmp/build.tcl
# set part_name "xc7a100tcsg324-1"
# cd /tmp/
# read_verilog "/tmp/counter.v"
# synth_design -top counter -part $part_name
Command: synth_design -top counter -part xc7a100tcsg324-1
Starting synth_design
Attempting to get a license for feature 'Synthesis' and/or device 'xc7a100t'
INFO: [Common 17-349] Got license for feature 'Synthesis' and/or device 'xc7a100t'
INFO: [Synth 8-7079] Multithreading enabled for synth_design using a maximum of 4 processes.
INFO: [Synth 8-7078] Launching helper process for spawning children vivado processes
INFO: [Synth 8-7075] Helper process

## arty

https://digilent.com/reference/programmable-logic/arty-a7/start

https://www.youtube.com/watch?v=Grs0gjeMPOY&ab_channel=AMDXilinx